In [1]:
import csv
from collections import deque

# Function to load a Turing Machine's definition from a CSV file
def load_from_file(file_name, states, input_alphabet, tape_alphabet, start_state,
                   accept_state, reject_state, transitions):
    """
    Reads the Turing Machine definition from a CSV file and initializes the required components.

    :param file_name: Name of the CSV file containing the TM definition.
    :param states: Set to store the states of the TM.
    :param input_alphabet: Set to store the input alphabet (Σ).
    :param tape_alphabet: Set to store the tape alphabet (Γ).
    :param start_state: Set to store the start state.
    :param accept_state: Set to store the accept state.
    :param reject_state: Set to store the reject state.
    :param transitions: Dictionary to store the transition function as {(state, symbol): [(next_state, write_char, move_dir)]}.
    """
    with open(file_name, 'r') as file:
        reader = csv.reader(file)
        lines = list(reader)
        # Load components from header lines
        states.update(lines[1][0].split(','))
        input_alphabet.update(lines[2][0].split(','))
        tape_alphabet.update(lines[3][0].split(','))
        start_state.update([lines[4][0]])
        accept_state.update([lines[5][0]])
        reject_state.update([lines[6][0]])
        # Load transitions from subsequent lines
        for line in lines[7:]:
            state, read_char, next_state, write_char, move_dir = line
            if (state, read_char) not in transitions:
                transitions[(state, read_char)] = []
            transitions[(state, read_char)].append((next_state, write_char, move_dir))

# Function to simulate the NTM and trace its behavior
def trace_tm_jklonosk(input_string, start_state, accept_state, reject_state,
                      transitions):
    """
    Simulates the behavior of a Non-deterministic Turing Machine (NTM) on a given input string.

    :param input_string: Input string to simulate on the TM.
    :param start_state: Start state of the TM.
    :param accept_state: Accept state of the TM.
    :param reject_state: Reject state of the TM.
    :param transitions: Transition function of the TM.
    :return: Tuple containing the result ("Accepted" or "Rejected") and the final tape configuration.
    """
    # Initialize the tape and configuration queue
    tape = list(input_string + '_')  # Add blank (_) to the tape
    queue = deque([(start_state.pop(), 0, tape, 0)])  # Start with initial configuration (state, head, tape, depth)
    max_depth = 0
    transition_count = 0

    # Breadth-first search for exploring all paths
    while queue:
        state, head, tape, depth = queue.popleft()  # Dequeue the current configuration
        max_depth = max(max_depth, depth)
        transition_count += 1

        # Check if the current configuration is an accept state
        if accept_state and state == accept_state.pop():
            output_result(input_string, "Accepted", tape, depth, transition_count)
            return "Accepted", tape

        # Ignore configurations that reach the reject state
        if reject_state and state == reject_state.pop():
            continue

        # Get the current tape symbol under the head
        tape_symbol = tape[head] if head < len(tape) else '_'

        # Process all possible transitions for the current state and tape symbol
        for next_state, write_char, move_dir in transitions.get((state, tape_symbol), []):
            # Create a new tape for this configuration
            new_tape = tape.copy()
            if head < len(new_tape):
                new_tape[head] = write_char  # Write to the tape at the head position
            else:
                new_tape.append(write_char)  # Append new character if head moves beyond the tape

            # Update the head position based on the direction
            new_head = max(0, head + (1 if move_dir == 'R' else -1))

            # Add the new configuration to the queue
            queue.append((next_state, new_head, new_tape, depth + 1))

    # If no accepting configuration is found, report rejection
    output_result(input_string, "Rejected", tape, max_depth, transition_count)
    return "Rejected", tape

# Function to output the result to a file
def output_result(input_string, result, tape, depth, transition_count):
    """
    Outputs the simulation result to a file specified by the user.

    :param input_string: The original input string.
    :param result: Result of the simulation ("Accepted" or "Rejected").
    :param tape: Final tape configuration.
    :param depth: Depth of the configuration tree.
    :param transition_count: Total number of transitions simulated.
    """
    output = input("Enter the name of the output file: ")  # Get output file name from user
    with open(output, 'w') as file:
        file.write(f"Input: {input_string}\n")
        file.write(f"Result: {result}\n")
        file.write(f"Depth: {depth}\n")
        file.write(f"Transition: {transition_count}\n")
        file.write(f"Final Tape: {''.join(tape)}\n")

# Main function to load the TM and simulate input strings
def main():
    """
    Main function to initialize the Turing Machine and simulate input strings.
    """
    # Initialize sets and dictionaries for TM components
    states = set()
    input_alphabet = set()
    tape_alphabet = set()
    start_state = set()
    accept_state = set()
    reject_state = set()
    transitions = {}

    # Prompting for the path to the CSV file
    file_path = input("Enter the name of the file: ")  # TM definition file
    load_from_file(file_path, states, input_alphabet, tape_alphabet, start_state,
                   accept_state, reject_state, transitions)

    # Test the Turing Machine with an example input string
    input_string = input("Enter the input string to simulate: ")
    result, final_tape = trace_tm_jklonosk(input_string, start_state, accept_state,
                                           reject_state, transitions)
    print(result, ''.join(final_tape))

if __name__ == "__main__":
    main()



KeyboardInterrupt: Interrupted by user